In [39]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np


# Read / Cleaning the data
social_spending = pd.read_csv("Data_cleaned/social_spending.csv", header=1)
gdp = pd.read_csv("Data_cleaned/gdp.csv")
gdp['gdp'] = gdp['gdp'].replace('..', np.nan)
gdp['gdp'] = gdp['gdp'].str.replace(',', '')
gdp['gdp'] = gdp['gdp'].astype(float)
social_spending = social_spending.rename(columns={'Year': 'Country'})
social_spending = social_spending.applymap(lambda x: x.strip() if isinstance(x, str) else x)
social_spending = pd.melt(social_spending, id_vars=['Country'], var_name='Year', value_name='s_s')
social_spending['Country_Year'] = social_spending['Country'] + '_' + social_spending['Year'].astype(str)

merged_data = pd.merge(gdp, social_spending, on=['Country_Year'])
merged_data['gdp'] = pd.to_numeric(merged_data['gdp'], errors='coerce')
merged_data['s_s'] = pd.to_numeric(merged_data['s_s'], errors='coerce')
#merged_data['gdp'] = merged_data['gdp'].fillna(merged_data['gdp'].mean())
#merged_data['s_s'] = merged_data['s_s'].fillna(merged_data['s_s'].mean())


average_social_spending = merged_data.groupby('Country')['s_s'].mean().reset_index()
average_gdp = merged_data.groupby('Country')['gdp'].mean().reset_index()
average_data = pd.merge(average_social_spending, average_gdp, on='Country', suffixes=('_s_s', '_gdp'))
average_data['s_s_gdp_ratio'] = average_data['s_s'] / average_data['gdp']

color_map = {
    'Australia': 'rgb(255, 0, 0)',        # Red
    'Austria': 'rgb(0, 255, 0)',          # Green
    'Belgium': 'rgb(0, 0, 255)',          # Blue
    'Canada': 'rgb(255, 255, 0)',         # Yellow
    'Chile': 'rgb(255, 0, 255)',          # Magenta
    'Colombia': 'rgb(0, 255, 255)',       # Cyan
    'Costa Rica': 'rgb(255, 127, 0)',     # Orange
    'Czechia': 'rgb(255, 0, 127)',        # Reddish-Pink
    'Denmark': 'rgb(0, 127, 255)',        # Bluish
    'Estonia': 'rgb(127, 255, 0)',        # Greenish-Yellow
    'Finland': 'rgb(0, 255, 127)',        # Greenish
    'France': 'rgb(127, 0, 255)',         # Purplish-Blue
    'Germany': 'rgb(255, 255, 127)',      # Light Yellow
    'Greece': 'rgb(127, 255, 255)',       # Light Cyan
    'Hungary': 'rgb(255, 127, 255)',      # Light Magenta
    'Iceland': 'rgb(255, 0, 0)',          # Red
    'Ireland': 'rgb(0, 255, 0)',          # Green
    'Israel': 'rgb(0, 0, 255)',           # Blue
    'Italy': 'rgb(255, 255, 0)',          # Yellow
    'Japan': 'rgb(255, 0, 255)',          # Magenta
    'Korea': 'rgb(0, 255, 255)',          # Cyan
    'Latvia': 'rgb(255, 127, 0)',         # Orange
    'Lithuania': 'rgb(255, 0, 127)',      # Reddish-Pink
    'Luxembourg': 'rgb(0, 127, 255)',     # Bluish
    'Mexico': 'rgb(127, 255, 0)',         # Greenish-Yellow
    'Netherlands': 'rgb(0, 255, 127)',    # Greenish
    'New Zealand': 'rgb(127, 0, 255)',    # Purplish-Blue
    'Norway': 'rgb(255, 255, 127)',      # Light Yellow
    'Poland': 'rgb(127, 255, 255)',       # Light Cyan
    'Portugal': 'rgb(255, 127, 255)',     # Light Magenta
    'Slovak Republic': 'rgb(255, 0, 0)',  # Red
    'Slovenia': 'rgb(0, 255, 0)',         # Green
    'Spain': 'rgb(0, 0, 255)',            # Blue
    'Sweden': 'rgb(255, 255, 0)',         # Yellow
    'Switzerland': 'rgb(255, 0, 255)',    # Magenta
    'Türkiye': 'rgb(0, 255, 255)',        # Cyan
    'United Kingdom': 'rgb(255, 127, 0)', # Orange
    'United States': 'rgb(255, 0, 127)',  # Reddish-Pink
}

average_data['color'] = average_data['Country'].map(color_map)
bubble_sizes = average_data['s_s_gdp_ratio'] * 1000

fig = go.Figure(data=go.Scatter(
    x=average_data['s_s_gdp_ratio'],
    y=average_data['gdp'],
    mode='markers',
    marker=dict(
        size=bubble_sizes,
        sizemode='area',
        sizeref=2.*max(bubble_sizes)/(100**2),
        sizemin=4,
        color=average_data['color']
    ),
    text=average_data.apply(lambda row: f"Country: {row['Country']}<br>Avg Social Spending: {row['s_s']:,}<br>Avg GDP: {row['gdp']:,}<br>Ratio: {row['s_s_gdp_ratio']:.4f}", axis=1),
))

fig.update_layout(
    title='Figure 3: Ratio of GDP Spent on Family Benefits',
    xaxis=dict(title='Average Percentage of GDP spent on Family Benefits'),  
    yaxis=dict(title='Average GDP (Billions) Over 20 Years'),
    showlegend=False
)

fig.show()

/var/folders/x3/1m329lgj7sl4y1pqkx_p_f_80000gn/T/ipykernel_58663/36842028.py:22: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.



In [35]:
us_data = average_data[average_data['Country'] == 'United States']
print("United States Results:")
print("Average Percentage Social Spending:", us_data['s_s'].values[0])
print("Average GDP:", us_data['gdp'].values[0])
print("Ratio of Average Social Spending to GDP:", us_data['s_s_gdp_ratio'].values[0])


United States Results:
Average Percentage Social Spending: 1.16
Average GDP: 15581.214210526316
Ratio of Average Social Spending to GDP: 7.444862668124607e-05


In [36]:
iceland_data = average_data[average_data['Country'] == 'Iceland']
print("Iceland Results:")
print("Average Percentage Social Spending:", iceland_data['s_s'].values[0])
print("Average GDP:", iceland_data['gdp'].values[0])
print("Ratio of Average Social Spending to GDP:", iceland_data['s_s_gdp_ratio'].values[0])

Iceland Results:
Average Percentage Social Spending: 3.2333333333333334
Average GDP: 13.97578947368421
Ratio of Average Social Spending to GDP: 0.23135246416108057
